In [2]:
!pip install pymysql


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!pip install cmake

   ---------------------------------------- 0.0/36.5 MB ? eta -:--:--
   -- ------------------------------------- 1.8/36.5 MB 14.3 MB/s eta 0:00:03
   ----- ---------------------------------- 4.7/36.5 MB 13.5 MB/s eta 0:00:03
   ------------ --------------------------- 11.0/36.5 MB 19.6 MB/s eta 0:00:02
   ------------------- -------------------- 18.1/36.5 MB 23.8 MB/s eta 0:00:01
   ------------------------- -------------- 23.3/36.5 MB 23.8 MB/s eta 0:00:01
   ------------------------------ --------- 27.8/36.5 MB 23.5 MB/s eta 0:00:01
   ---------------------------------- ----- 31.5/36.5 MB 22.4 MB/s eta 0:00:01
   ---------------------------------------  35.7/36.5 MB 22.2 MB/s eta 0:00:01
   ---------------------------------------- 36.5/36.5 MB 21.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
!pip install dlib

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.6-cp312-cp312-win_amd64.whl size=2919815 sha256=7f3fd950c81b46563e5202215e4c1120f04d932f45dc8bf571b80b26dab75a10
  Stored in directory: c:\users\chari\appdata\local\pip\cache\wheels\8d\16\b2\d2df2ea5d9430e80343c7b696e9588e5cc98b0d903d434d768
Successfully built dlib



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pymysql
import json
import numpy as np
from pymysql.constants import CLIENT

### Establishing database connection

In [6]:
HOST = 'localhost'
USER = 'root'
PASSWORD = 'Cherry@10082001'
CHARSET = 'utf8mb4'
DBNAME = 'students'
PORT = 3306

### Inserting person face into database

In [7]:
def createDatabase():
    
    conn = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        port=3306,
        cursorclass=pymysql.cursors.DictCursor,
        client_flag=CLIENT.MULTI_STATEMENTS
    )

    sql = "\n".join([
    f"DROP SCHEMA IF EXISTS `{DBNAME}`;",
    f"CREATE SCHEMA IF NOT EXISTS `{DBNAME}` DEFAULT CHARACTER SET utf8;",
    "SHOW WARNINGS;",
    f"USE `{DBNAME}`;",
    f"DROP TABLE IF EXISTS `{DBNAME}`.`student_face_encodings`;",
    "SHOW WARNINGS;",
    f"CREATE TABLE IF NOT EXISTS `{DBNAME}`.`student_face_encodings` (",
    "    `id` INT NOT NULL AUTO_INCREMENT,",
    "    `name` NVARCHAR(255) NOT NULL,",
    "    `faceencoding` TEXT NOT NULL,",
    "    PRIMARY KEY (`id`)",
    ") ENGINE = InnoDB;"])

    sql = sql.replace("\n", "")
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql)
            conn.commit()

            sql_show = "SHOW DATABASES"
            cursor.execute(sql_show)
            databaseList = cursor.fetchall()
            for database in databaseList:
                if database['Database'] == DBNAME:
                    print(database, 'has been successfully created!')
                    break

    except Exception as e:
        print("Database creation failed: {}".format(e))

    finally:
        conn.close()

In [8]:
createDatabase()

{'Database': 'students'} has been successfully created!


In [9]:
def insertFaceToDatabase(name, encoding):
    
    conn = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DBNAME,
        port=3306,
        cursorclass=pymysql.cursors.DictCursor
    )

    try:
        with conn.cursor() as cursor:
            sql = "INSERT INTO student_face_encodings (name, faceencoding) VALUES ('{}', '{}')".format(
                name, json.dumps(encoding))
            cursor.execute(sql)
            conn.commit()
            print("Insertion Success!")

    except Exception as e:
        conn.rollback()
        print('Insertion Failed!', e)
    conn.close()

In [10]:
def fetchAllStudents():
    
    conn = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DBNAME,
        port=PORT,
        cursorclass=pymysql.cursors.DictCursor)

    try:
        with conn.cursor() as cursor:
            sql = "SELECT name, faceencoding FROM student_face_encodings"
            cursor.execute(sql)
            data = cursor.fetchall()
            encodings, names = [], []

            for i in range(0, len(data)):
                names.append(data[i]['name'])
                encodings.append(json.loads(data[i]['faceencoding']))

            print("Fetching Success!")

    except Exception as e:
        conn.rollback()
        print('Fetching Failed:\n', e)

    conn.close()
    return encodings, names

In [11]:
def testInsertion():
    data = [-0.06413651,  0.05275547,  0.11465468, -0.06868585, -0.12905313, -0.06219113, -0.08105582, -0.05795766,  0.14002423, -0.14891383, 0.16809663,  0.03969061, -0.24029616,  0.01072336,  0.01820949, 0.20016633, -0.11752109, -0.12041475, -0.0686784,  0.02890174, 0.01413991, -0.00092969, -0.00206049,  0.05478341, -0.07208329, -0.38025042, -0.09110465, -0.03197921, -0.04670967,  0.00347318, -0.1002866,  0.02906058, -0.1258755, -0.06500086,  0.08338613, 0.14355671, -0.08919339, -0.08197339,  0.20867996,  0.01007678, -0.2455171, -0.03276137,  0.07985185,  0.2051305,  0.24185342, 0.0678556,  0.08049953, -0.09811792,  0.18532458, -0.34516171, 0.03625763,  0.09210046,  0.05877451,  0.06729918,  0.12198181, -0.13268434,  0.0351983,  0.13155086, -0.17172278, -0.04020822, 0.00662367, -0.09220872, -0.03645555, -0.07874616, 0.27311197, 0.1041171, -0.13304791, -0.09074535,  0.16285834, -0.18031871, -0.09120225,  0.01239933, -0.09920017, -0.15501264, -0.33115372, -0.01934731,  0.39288402,  0.15537588, -0.15778066,  0.06874227, -0.0653723, -0.03062057,  0.07541025,  0.05756121, -0.02222584, 0.01667677, -0.08238429, -0.01497209,  0.27893612, -0.08270567, 0.05658919,  0.18633389,  0.03200752,  0.08384499,  0.0460855, 0.0695291, -0.09776485, -0.05492003, -0.22801645, -0.05175924, 0.05780323, -0.02099415,  0.0295985,  0.10740275, -0.26527831, 0.09936604, -0.03858019, -0.04595775,  0.04024224, -0.01442718, -0.16451956, -0.08263291,  0.13128407, -0.21504368,  0.14097233, 0.18843487, -0.03240553,  0.11317647,  0.10367718,  0.0171592, -0.02866501, -0.07389878, -0.15731311, -0.09558592,  0.10734915, 0.03525282,  0.05941724,  0.0096033]
    
    insertFaceToDatabase(name='Cherry', encoding=data)

In [12]:
testInsertion()

Insertion Success!


In [13]:
def testRead():
    encodings, names = fetchAllStudents()

    print('\n\n\n\n', type(encodings), encodings, '\n\n', names)

In [14]:
testRead()

Fetching Success!




 <class 'list'> [[-0.06413651, 0.05275547, 0.11465468, -0.06868585, -0.12905313, -0.06219113, -0.08105582, -0.05795766, 0.14002423, -0.14891383, 0.16809663, 0.03969061, -0.24029616, 0.01072336, 0.01820949, 0.20016633, -0.11752109, -0.12041475, -0.0686784, 0.02890174, 0.01413991, -0.00092969, -0.00206049, 0.05478341, -0.07208329, -0.38025042, -0.09110465, -0.03197921, -0.04670967, 0.00347318, -0.1002866, 0.02906058, -0.1258755, -0.06500086, 0.08338613, 0.14355671, -0.08919339, -0.08197339, 0.20867996, 0.01007678, -0.2455171, -0.03276137, 0.07985185, 0.2051305, 0.24185342, 0.0678556, 0.08049953, -0.09811792, 0.18532458, -0.34516171, 0.03625763, 0.09210046, 0.05877451, 0.06729918, 0.12198181, -0.13268434, 0.0351983, 0.13155086, -0.17172278, -0.04020822, 0.00662367, -0.09220872, -0.03645555, -0.07874616, 0.27311197, 0.1041171, -0.13304791, -0.09074535, 0.16285834, -0.18031871, -0.09120225, 0.01239933, -0.09920017, -0.15501264, -0.33115372, -0.01934731, 0.39288402, 0.1

In [15]:
from PIL import Image

In [16]:
!pip install face_recognition


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import face_recognition

In [18]:
def insertNewPersonWithImage():
    image_path = input("Please Enter Image Path: ")
    name = input("Person name: ")
    img = ''
    encode = []
    try:
        img = face_recognition.load_image_file(image_path)
        encode = face_recognition.face_encodings(img)[0]
        list = encode.tolist()
        insertFaceToDatabase(name, list)
        print(name, " Added successfully!")
    except Exception as e:
        print("Failed to load image: {}".format(e))

In [19]:
insertNewPersonWithImage()

Please Enter Image Path:  Charitha.jpg
Person name:  Charitha


Insertion Success!
Charitha  Added successfully!


In [38]:
insertNewPersonWithImage()

Please Enter Image Path:  samantha.jpg
Person name:  Samantha


Insertion Success!
Samantha  Added successfully!


### Eye Tracking

In [41]:
!pip install opencv-python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 4.2 MB/s eta 0:00:10
   --- ------------------------------------ 3.4/39.5 MB 11.2 MB/s eta 0:00:04
   ------- -------------------------------- 7.6/39.5 MB 14.7 MB/s eta 0:00:03
   -------------- ------------------------- 14.4/39.5 MB 19.7 MB/s eta 0:00:02
   --------------------- ------------------ 21.5/39.5 MB 22.7 MB/s eta 0:00:01
   ------------------------- -------------- 25.4/39.5 MB 21.8 MB/s eta 0:00:01
   ---------------------------- ----------- 28.3/39.5 MB 20.6 MB/s eta 0:00:01
   ------------------------------- -------- 31.5/39.5 MB 19.6 MB/s eta 0:00:01
   ---------------------------------- ----- 34.1/39.5 MB 19.0 MB/s eta 0:00:01
   ------------------------------------- -- 37.2/39.5 MB 18.5 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.5 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 16.8 MB/


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import cv2
import dlib
import numpy as np

In [21]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

In [25]:
def eye_on_mask(mask, side):  
    points = [shape[i] for i in side]
    points = np.array(points, dtype=np.int32)
    mask = cv2.fillConvexPoly(mask, points, 255)
    return mask

In [26]:
def contouring(thresh, mid, img, right=False):
    cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    try:
        cnt = max(cnts, key=cv2.contourArea)  
        M = cv2.moments(cnt)  
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        if right:
            cx += mid  
        cv2.circle(img, (cx, cy), 1, (0, 255, 0), 3)  
        return cy, cx
    except:
        pass  

In [27]:
def compare_eyeballs(y_left, x_left, y_right, x_right, y_center, x_center, img):
    try:
        diff_x = abs(x_left - x_right) / 4
        cv2.putText(img, 'left : ' + str(x_left) + "  center : " + str(x_center) + "  right : " + str(x_right),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
        img1 = cv2.imread('cheating.jpeg')
        cv2.imshow("eyes", img)
        if x_center - diff_x <= x_left:
            cv2.putText(img1, "Dont look left",(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
            cv2.imshow('cheater image looking left', img1)
            cv2.waitKey(0)
        elif x_center + diff_x >= x_right:
            cv2.putText(img1, "Dont look Right",(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
            cv2.imshow('cheater image looking right', img1)
            cv2.waitKey(0)
        else:
            cv2.destroyWindow('cheater image')
    except:
        pass

In [28]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_68.dat')

left = [36, 37, 38, 39, 40, 41]  # keypoint indices for left eye
right = [42, 43, 44, 45, 46, 47]  # keypoint indices for right eye

RuntimeError: Unable to open shape_68.dat

In [29]:
cap = cv2.VideoCapture(0)
ret, img = cap.read()
thresh = img.copy()
kernel = np.ones((20, 20), np.uint8)
process_frame = False
while (True):
    image2 = np.zeros((1000, 1000, 3), np.uint8)
    ret, img = cap.read()
    if not process_frame:
        process_frame = True
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)
        for rect in rects:
            shape = predictor(gray, rect)  
            shape = shape_to_np(shape)  
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            mask = eye_on_mask(mask, left)  
            mask = eye_on_mask(mask, right)
            mask = cv2.dilate(mask, kernel, 5)  
            eyes = cv2.bitwise_and(img, img, mask=mask)  
            mask = (eyes == [0, 0, 0]).all(axis=2)
            eyes[mask] = [255, 255, 255]
            mid = (shape[42][0] + shape[39][0]) // 2
            eyes_gray = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
            threshold = 67
            #threshold = cv2.getTrackbarPos('threshold', 'image')
            _, thresh = cv2.threshold(eyes_gray, threshold, 255, cv2.THRESH_BINARY)
            thresh = cv2.erode(thresh, None, iterations=2)  # 1
            thresh = cv2.dilate(thresh, None, iterations=4)  # 2
            thresh = cv2.medianBlur(thresh, 3)  # 3 smoothing the image
            thresh = cv2.bitwise_not(thresh)  # to find the eyeballs it need to be white and its background black
            eyeLeft = contouring(thresh[:, 0:mid], mid, img)
            eyeRight = contouring(thresh[:, mid:], mid, img, True)

            try:
                compare_eyeballs(shape[36][1], shape[36][0], shape[39][1], shape[39][0], eyeLeft[0], eyeLeft[1],
                                 img)
            except:
                pass

            try:
                compare_eyeballs(shape[42][1], shape[42][0], shape[45][1], shape[45][0], eyeRight[0], eyeRight[1],
                                 img)
            except:
                pass

        process_frame = False

    cv2.imshow('eyes', img)
    #cv2.imshow("image", thresh)
    # cv2.destroyWindow('image')
    #cv2.destroyWindow('eyePoints')

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'

### Face Recognition, Multi-face Detection and Cell-Phone, Textbook detection

##### Using MobileNet model to detect cell phone and textbook

In [30]:
def face_recog(frame, known_faces_encodings: list, names_students: list):

        names = []
        frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_frame =  np.ascontiguousarray(frame[:, :, ::-1])
        encodings = face_recognition.face_encodings(rgb_frame)
        for unknown_face_encoding in encodings:
            matches = face_recognition.compare_faces(known_faces_encodings, unknown_face_encoding)
            name = "Unknown"
            if True in matches:
                match_index = matches.index(True)
                name = names_students[match_index]
            names.append(name)

        if len(names) == 0:
            names = ["No face is visible"]

        return names

In [31]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [32]:
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

In [33]:
classes = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor", "cellphone", "book"]

In [34]:
def detect_objects(frame):
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), (127.5, 127.5, 127.5), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward()

    detected_objects = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:  # Confidence threshold
            class_id = int(detections[0, 0, i, 1])
            if class_id == 21:  # "cellphone" class
                detected_objects.append("cellphone")
            elif class_id == 22:  # "book" class
                detected_objects.append("book")
    return detected_objects


In [42]:
class VideoCap:

    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        self.encodings, self.student_names = fetchAllStudents()
        (self.grabbed, self.frame) = self.stream.read()
        self.names = ["unknown"]
        self.stopped = False

    def get_and_show(self):
        no_of_frames = 0
        count_multi = 0

        while not self.stopped:
            if (not self.grabbed) or cv2.waitKey(1) == ord("q"):
                self.stop()
            else:
                (self.grabbed, self.frame) = self.stream.read()
                no_of_frames += 1
                if no_of_frames % 60 == 0:
                    self.names = face_recog(self.frame, self.encodings, self.student_names)
                    # using locations instead of encodings because it's faster 
                    # detect how many faces 
                    locations = face_recognition.face_locations(self.frame)
                    # if more than one then it's group cheating
                    if len(locations) > 1:
                        count_multi += 1
                        print(count_multi)
                        img = cv2.imread("warning.png")
                        cv2.imshow("Multiple faces detected", img)
                    else:
                        # Detect faces in the frame
                        gray = cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
                        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

                        # Process each detected face
                        for (x, y, w, h) in faces:
                            roi_gray = gray[y:y + h, x:x + w]                      

                # Detect objects (cellphones and books)
                detected_objects = detect_objects(self.frame)
                for obj in detected_objects:
                    if obj == "cellphone":
                        cv2.putText(self.frame, "Cellphone Detected!", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        img = cv2.imread("warning.png")
                        cv2.imshow("Cellphone Detected", img)
                    elif obj == "book":
                        cv2.putText(self.frame, "Book Detected!", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        img = cv2.imread("warning.png")
                        cv2.imshow("Book Detected", img)

            cv2.putText(self.frame, " ,".join(self.names), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 1)
            cv2.imshow("Live Video", self.frame)

            if count_multi == 3:
                print("Multiple faces exceeded")
                img = cv2.imread("fAILED.jpg")
                cv2.imshow("Multiple faces detected", img)
                cv2.waitKey(5000)
                self.stop()
    def stop(self):
        self.stream.release()
        cv2.destroyAllWindows()
        self.stopped = True


In [43]:
video = VideoCap().get_and_show()

Fetching Success!


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
